In [1]:
!pip install qiskit qiskit-machine-learning==0.6.1 qiskit-aer torch torchvision --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 798.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.0 MB/s eta 0:00:00
   ━━

In [1]:
from google.colab import files
files.upload()  # Upload kaggle.json here

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"ananyarachar","key":"ed9a2181fb870f1624c45696e52965ac"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the PlantVillage dataset
!kaggle datasets download -d mohitsingh1804/plantvillage
!unzip -q plantvillage.zip -d plantvillage

Dataset URL: https://www.kaggle.com/datasets/mohitsingh1804/plantvillage
License(s): GPL-2.0
plantvillage.zip: Skipping, found more recently modified local copy (use --force to force download)
replace plantvillage/PlantVillage/train/Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78752cd043f___FREC_Scab 3003.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [6]:
!pip install qiskit==1.4.2 qiskit-machine-learning==0.7.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.0.0
    Uninstalling qiskit-2.0.0:
      Successfully uninstalled qiskit-2.0.0
  Attempting uninstall: qiskit-machine-learning
    Found existing installation: qiskit-machine-learning 0.8.2
    Uninstalling qiskit-machine-learning-0.8.2:
      Successfully uninstalled qiskit-machine-learning-0.8.2


In [5]:
# Import necessary libraries
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

import torch
import torch.nn as nn
import torch.optim as optim

from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Step 1: Load and preprocess the dataset
selected_classes = ['Tomato___Bacterial_spot', 'Tomato___healthy',
                    'Strawberry___Leaf_scorch', 'Corn_(maize)___Common_rust_',
                    'Peach___Bacterial_spot', 'Peach___healthy', 'Potato___healthy']
image_dir = 'plantvillage/PlantVillage/train'  # Adjust this path

X, y = [], []

for class_name in selected_classes:
    class_path = os.path.join(image_dir, class_name)
    for img_name in tqdm(os.listdir(class_path)[:100]):  # Limit to 100 images per class
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            hist = cv2.calcHist([img], [0, 1, 2], None, [2, 2, 2],
                                [0, 256, 0, 256, 0, 256])  # 2x2x2 = 8 features
            X.append(hist.flatten())
            y.append(class_name)

X = np.array(X)
y = np.array(y)

print(f"Feature shape: {X.shape}, Labels: {np.unique(y)}")

# Step 2: Normalize and encode labels
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

y_encoded = LabelEncoder().fit_transform(y)

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42)

# Step 4: Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Step 5: Create variational quantum circuit
num_qubits = X_train_tensor.shape[1]
ansatz = TwoLocal(num_qubits, rotation_blocks='ry', entanglement_blocks='cz', reps=1)

# Step 6: Create EstimatorQNN
qnn = EstimatorQNN(
    circuit=ansatz,
    input_params=ansatz.parameters[:num_qubits],
    weight_params=ansatz.parameters[num_qubits:]
)

# Step 7: Connect QNN with PyTorch
model = TorchConnector(qnn)

# Step 8: Define the Quantum Classifier model
class QuantumClassifier(nn.Module):
    def __init__(self, model):
        super(QuantumClassifier, self).__init__()
        self.qnn = model
        self.classifier = nn.Linear(1, len(np.unique(y)))  # Output layer

    def forward(self, x):
        q_out = self.qnn(x)
        return self.classifier(q_out)

net = QuantumClassifier(model)

# Step 9: Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)
epochs = 5

# Step 10: Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    output = net(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Step 11: Evaluate the model
with torch.no_grad():
    preds = net(X_test_tensor)
    predicted = torch.argmax(preds, axis=1)
    acc = accuracy_score(y_test_tensor, predicted)
    print(f"\nTest Accuracy (QNN): {acc:.4f}")
    print("\nClassification Report:\n", classification_report(y_test_tensor, predicted))


100%|██████████| 100/100 [00:00<00:00, 291.88it/s]


Feature shape: (700, 8), Labels: ['Corn_(maize)___Common_rust_' 'Peach___Bacterial_spot' 'Peach___healthy'
 'Potato___healthy' 'Strawberry___Leaf_scorch' 'Tomato___Bacterial_spot'
 'Tomato___healthy']
Epoch 1/5, Loss: 2.0357
Epoch 2/5, Loss: 2.0298
Epoch 3/5, Loss: 2.0241
Epoch 4/5, Loss: 2.0184
Epoch 5/5, Loss: 2.0129

Test Accuracy (QNN): 0.1000

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.12      0.93      0.21        14
           2       0.00      0.00      0.00        18
           3       0.00      0.00      0.00        20
           4       0.03      0.05      0.04        21
           5       0.00      0.00      0.00        25
           6       0.00      0.00      0.00        22

    accuracy                           0.10       140
   macro avg       0.02      0.14      0.04       140
weighted avg       0.02      0.10      0.03       140



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
# Save the trained model
model_save_path = 'qnn_model.pth'
torch.save(net.state_dict(), model_save_path)

print(f"Model saved to {model_save_path}")


Model saved to qnn_model.pth
